# [译]音乐信息获取(MIR)之学习对音频的节拍追踪(Beat Tracking)
原文出处：[audio-beat-tracking-for-music-information-retrieval](https://www.analyticsvidhya.com/blog/2018/02/audio-beat-tracking-for-music-information-retrieval/)
本文在忠实原文的基础上补充了部分内容（如绘图，参考文献等）


## 介绍
音乐无处不在。当我们听到那种打动人心的音乐，我们就会整个沉浸在其中。同时，我们随着听到的节奏而打着拍子。你也一定注意到过你的腿不由自主的随着音乐的节拍而抖动。在这方面没有任何逻辑上可以解释我们为何会这样做，可能仅仅是因为我们沉浸在了音乐的律动中，我们的大脑开始对这种曲调产生共鸣。

![img-music](./res/img-music.jpg)

假如我们能训练一个人工系统像我们人一样来捕捉这样的律动呢？可以想象，一个很酷的应用可能是构建一个具有表现力的类人机器人，它运行实时的节拍跟踪算法，使其在跳舞时能够与音乐保持同步。

![robot-dancing](./res/robot-dancing.gif)
原视频地址：[DARwin-OP dancing with beat tracking](https://www.youtube.com/watch?v=AJ--LrnkR6Y)

很有趣，对吧？

在本文中，我们将了解节拍（beats）的概念，以及在跟踪它时所面临的挑战。然后我们将学习解决这些问题的方法，和行业内最先进的解决方案。

注意:本文假设您具有python中音频数据分析的基本知识。如果没有，你可以阅读[这篇文章](https://www.analyticsvidhya.com/blog/2017/08/audio-voice-processing-deep-learning/)，然后继续阅读。

## 目录
- 节拍追踪概览
  - 什么是节拍追踪？
  - 节拍追踪的应用
  - 节拍追踪中的挑战
- 解决节拍追踪的方法
  - 动态规划
  - 循环神经网络 + 深度置信网
  
## 节拍追踪概览

### 什么是节拍追踪（Beat Tracking）
音频节拍追踪通常定义为确定音频记录中的时间实例，人类听众可能会随着音乐轻拍自己的脚。音频节拍跟踪技术使音乐的“拍同步”分析成为可能。

作为音乐信息获取（MIR）的一项重要和相关的任务，这一领域的研究十分活跃。自动节拍跟踪任务的目标是跟踪声音文件集合中的所有节拍位置，并为每个文件输出这些节拍开始时间。

为了给你一个对此任务的直观感受，下载并聆听下面的音频：

[原始音频](./res/teasure-trimed.wav)

[添加节拍注释的音频](./res/teasure-trimed-annotated.wav)


### 节拍追踪的应用
> 下面列举几个例子

1. 灯光随音乐节奏变化
2. Music-driven content的需要。音乐驱动的创作内容，如拍摄的视频短片，现在流行的一些音乐视频软件等，在融合音乐的时候都需要对节拍进行卡点。
3. 作为音乐分类的重要特征，等。


### 节拍追踪中的挑战
节拍跟踪可能听起来像一个简单的概念，但直到现在它实际上还是一个未解决的问题。 对于简单的曲调，算法可以很容易地从音乐中找出节拍。 但通常在现实生活中，音频要复杂得多，噪音也很大。 例如，可能存在来自环境的噪声，其可能会混淆算法并导致其在检测节拍时产生误报(False positive)。
从技术上讲，处理节拍跟踪有三个主要挑战：
1. 脉冲水平不清晰
2. 突然的节奏改变
3. 模糊/嘈杂的信息

## 节拍追踪中的可行方法
现在我们知道了一些有关节拍追踪的一些要点，下面我们来了解一下用来解决这个问题的一些方法。

音乐信息检索(music information retrival)算法的年度评估活动和ISMIR会议一起被称作音乐信息检索评估交换(MIREX)。其中有一个叫做音频街拍追踪的任务，研究人员参加MIREX并提交他们的方法，下面将介绍两种方法，第一种是简单而原始的，第二种是最先进的方法。

### 方法1: 突发点(onset)检测与动态规划
例如我们有下面的一段音频。
![original-image](./res/audio-original.png)

我们可以找到那种声音的突发点(也叫做onset)的位置，并且标注这些时间点，得到
![processed-image](./res/audio-processed.png)

这很可能是节拍的表现形式。 但它会包含许多误报(False positive)，例如人的声音或背景噪音。 因此，为了最大限度地减少这些误报，我们可以找到这些起始点的**最长公共子序列**来识别节拍。 如果您想了解动态编程的工作原理，可以参考[这篇文章](https://www.analyticsvidhya.com/blog/2016/05/ase-studies-10x-faster-using-dynamic-programming/)。

下面的代码实现让你有一个更清晰的认识。

In [1]:
import librosa
import IPython.display as ipd

# read audio file
y, sr = librosa.load('./res/treasure-trimed.wav')
ipd.Audio(y, rate=sr)

In [2]:
# method 1 - onset detection and dynamic programming
tempo, beat_times = librosa.beat.beat_track(y, sr=sr, units='time')

clicks = librosa.clicks(beat_times,sr=sr,length=len(y))
ipd.Audio(y + clicks, rate=sr)

- 参考文献

> [1] Ellis, Daniel PW. “Beat tracking by dynamic programming.” Journal of New Music Research 36.1 (2007): 51-60. http://labrosa.ee.columbia.edu/projects/beattrack/

## 方法2: 集成循环神经网络(RNN)和动态贝叶斯网络
> 我们应该更多关注这部分

我们可以使用机器学习/深度学习方法，而不是手动地依赖声音的提示。 下面展示了用于解决节拍跟踪的框架的体系结构。 要了解它的细节，你可以阅读官方研究论文。

方法的要点是 - 我们预处理音频信号，然后使用递归神经网络找出这些节拍时间的最可能的值。 研究人员还使用了一系列循环神经网络（RNN），然后使用贝叶斯网络对其输出进行整合。如下图所示：
![model](./res/rnn-bn-model.png)

madmom库包含在节拍跟踪领域中各种最先进算法的实现，可在[github](https://github.com/CPJKU/madmom)上获得源码。 它结合了基于机器学习方法的低级特征提取和高级特征分析。 方法2的代码可以从此仓库中获得，其实现了从输入音频文件输出节拍位置。

下面我们看一下方法2的代码实现。

In [3]:
import madmom

# method 2 - dbn tracker

proc = madmom.features.beats.DBNBeatTrackingProcessor(fps=100, min_bpm=60)
act = madmom.features.beats.RNNBeatProcessor()('./res/treasure-trimed.wav')

beat_times = proc(act)
print(beat_times)
clicks = librosa.clicks(beat_times, sr=sr, length=len(y))
ipd.Audio(y + clicks, rate=sr)

/Users/kevin/.virtualenvs/py36/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


[0.25 0.77 1.28 1.8  2.32 2.84 3.35 3.87 4.39 4.91 5.42]


- 参考文献

> [1] Sebastian Böck and Markus Schedl, “Enhanced Beat Tracking with Context-Aware Neural Networks”, Proceedings of the 14th International Conference on Digital Audio Effects (DAFx), 2011.
>
> [2] Florian Krebs, Sebastian Böck and Gerhard Widmer, “An Efficient State Space Model for Joint Tempo and Meter Tracking”, Proceedings of the 16th International Society for Music Information Retrieval Conference (ISMIR), 2015.

下面我们分析下方法2的处理步骤:

#### 1. 预处理音频信号

![model1](./res/model1.png)
与所有非结构化数据一样，人工系统不容易掌握音频信号的概念。 因此必须以一种可以解释为机器学习模型的格式（即预处理数据）进行转换。 如下图所示，你可以使用很多方法来预处理音频数据。
![model2](./res/model2.png)

1. 时域特征，如：波形的RMSE(均方根误差)
2. 频域特征，如：单个频率振幅
3. 感知特征，如：MFCC(梅尔频率倒谱系数)
4. 窗口特征，如：窗口的汉明距离

#### 2. 训练和微调RNN模型

![model3](./res/model3.png)
现在你已经预处理了数据，你可以应用机器学习/深度学习模型来学习到数据中的模式。

从理论上讲，如果你训练一个具有足够训练样例和适当架构的深度学习模型，它可以很好地解决问题。 但由于训练数据可用性不足等原因，这并非总是可行的。因此，为了提高性能，我们可以做的是在**单一类型的音乐文件上训练多个RNN模型**，以便它可以捕获模式的这种类型(genre)本身。 这有助于缓解一些数据不足问题。

对于这种方法，我们首先训练LSTM模型（RNN的改进版本）并将其设置为我们的基本模型。 然后我们微调从我们的基础模型派生的多个LSTM模型。 这种微调是在不同类型的音乐序列上完成的。 在测试时，我们传递来自所有模型的音频信号。

#### 3. 选择最佳的RNN模型

![model4](./res/model4.png)

在这一步，我们只是选择所有模型中误差最小的模型，将它们与我们从基础参照模型得到的分数进行比较。

#### 4. 应用动态贝叶斯网络

![model5](./res/model5.png)

无论你使用一个LSTM还是多个LSTM，都存在一个根本性的缺点：在选择节拍的最终位置时，最终的峰值发现阶段并不试图找到全局最优值。它确定了作品（或一定长度的片段）的主要节奏，然后根据这个节奏排列节拍位置，简单地选择最好的起始位置，然后逐步地将节拍定位在预先确定的位置周围某个区域的激活函数值最高的位置。

为了避免这个问题，我们将所选神经网络模型的输出馈送到动态贝叶斯网络（DBN）中，该网络共同推断节拍序列的节奏和相位。使用DBN的另一个优点是我们能够对节拍状态和非节拍状态进行建模，这表明其性能优于仅对节拍状态进行建模的情况。我不会详细介绍DBN的工作原理，但如果您有兴趣，可以参考[此视频](https://youtu.be/lecy8kEjC3Q)。

这就是我们如何从原始音频信号中获得音乐序列的节拍的大致方法。

## 结语

我希望这篇文章能让你直观地了解如何在python中解决节拍跟踪问题。 这在音乐信息检索领域具有潜力的应用，我们可以使用我们在跟踪检测它时获得的节拍，来识别相似类型的音乐。